# Main | LLM to Vocabulary


**Request Documentation**: https://platform.openai.com/docs/api-reference/completions/create

**OpenAI Documentation**: https://platform.openai.com/docs/quickstart?context=python

**Find your keys here**: https://platform.openai.com/api-keys

**Keep an eye on your credit usage**: https://platform.openai.com/settings/organization/usage

## OpenAI API Details and Hyper-Parameters

**Models**: The model you want to use (i.e. "gpt-4-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini" or "gpt-3.5-turbo")

**Message**: The message being sent (the prompt)

**Temperature**: Number between 0 and 1, can be adjusted when calling the OpenAI API to control the randomness of the output generated by the language model. It determines how "creative" or "conservative" the responses are. A temperature of 1.0 means the model generates text with standard randomness, while a lower temperature (closer to 0.0) will make it more deterministic. Suggestion: Set the temperature to a moderate value, around 0.2 to 0.5. This will give you some randomness, mimicking the variability in human input, but still maintain enough accuracy to ensure that the responses are meaningful. **In the case of our work, we held this constant at a value of 0.2 to get slight stochasticity.** 

**Frequency_penalty**: Number between -2 and 2, where higher numbers penalize new tokens based on their frequency to that point in the response. The higher the number, the lower the probability of repetition. This parameter penalizes repeated tokens in the output, making responses more diverse and creative. Suggestion: Use a low frequency penalty (e.g., 0.0 to 0.2) to reduce redundancy but still allow the model to use relevant tokens multiple times where necessary. **In the case of our work, we held this constant at a value of 0.0.**

**Top-p (Nucleus Sampling)**: This parameter determines the probability mass considered for each token in the output. A lower top-p (e.g., 0.7) will cause the model to sample from only the more likely tokens, while a top-p of 1.0 will consider all possible outcomes. Suggestion: Set the top_p to 0.8–0.9 to introduce some controlled randomness while still keeping the majority of the likelihood in more probable tokens, ensuring coherent responses. ~~~~~  Higher top_p values (closer to 1): This will include a broader range of possible tokens, increasing diversity and randomness, making the behaviour more stochastic (i.e., less predictable). A value of top_p = 1 means the model can sample from the full probability distribution, leading to more creative and unpredictable results.
Lower top_p values (closer to 0): This restricts the sampling to a smaller subset of tokens (those with the highest probability), leading to more deterministic and conservative outputs, i.e., less stochastic behavior. For example, setting top_p = 0.1 would mean the model will only consider the most probable 10% of tokens.

## Imports

In [1]:
%pwd

'/Users/liamroy/Documents/Studies/Monash_31194990/PHD/Studies/Study_04/LLM_vocab_optimization/scripts'

In [2]:
# Imports
import sys
import os
import importlib

import numpy as np
import random
import time
random.seed(time.time())

from itertools import product

from openpyxl import load_workbook

from openai import OpenAI
client = OpenAI()

from robots_and_modules.helper_functions import llm_prompt_reply
from robots_and_modules.prompt_builder import build_prompt
from robots_and_modules.distance_calculator import calculate_distance

## Define Inputs

In [3]:
## Configuration parameters
attempt_ID = '03'

iteration_quantity = 20
iteration_quantity_str = str(iteration_quantity).zfill(3)

llm_model = "gpt-4o"                    # gpt-3.5-turbo     | Use this for dev/testing
                                        # gpt-4 / gpt-4o    | Use this when deployed, more expensive
                                        # gpt-4o-mini       | Lightweight, less expensive than gpt4o
temperature_coefficient = 1.0           # Moderately stochastic @ 0.6 to 1.0
frequency_penalty_coefficient = 1.0     # Lightly penalize repetition @ 0.2
top_p_coefficient=1.0                   # Nucleus sampling for controlled randomness @ 0.85 to 0.6
omission_probability = 0.4

# Default to go1_obj and 'spread' state space
robot_string = 'jackal'     # 'jackal' or 'go1'
state_space = 'minimized'      # 'spread' or 'minimized'

printer = "partial"         # "all", "partial" or None

In [4]:
# Define the save string for the raw numpy array
normalized_accuracy_array_filename = robot_string + "_" + state_space + "_" + llm_model + "_" + attempt_ID + "ID_" + iteration_quantity_str + "ITR_norm.npy"
normalized_accuracy_array_save_string = f"./../data/acc_arrays/{normalized_accuracy_array_filename}"

transformed_trajectory_save_string = "../data/freedog/data/go1_transformed_trajectory_array.npy"


### Check Robot Module and State Space

In [5]:
# Check if robot_string and state_space are within the bounds of go1/jackal and spread/minimized
if robot_string not in ['go1', 'jackal']:
    print(f"Error: Invalid robot module specified. Using default: {robot_string}")
if state_space not in ['spread', 'minimized']:
    print(f"Error: Invalid state space specified. Using default: {state_space}")

# Dynamically import the module
robot_module = importlib.import_module(f"robots_and_modules.{robot_string}")

### Select State Space based on Module

In [6]:
# The data in this should be in the format: "State Number: [State Name, State Description]"
# Used for Go1 robot
if robot_string == 'go1':
    if state_space == 'spread':
        set_of_states = [
        "S01: [Waiting for Input, The robot is in standby mode, waiting for a command from the user.]",
        "S02: [Analyzing Object, The robot is analyzing a target object in front of it on the ground.]", 
        "S03: [Found Object, The robot is signalling to the user it has found a target object on the ground.]",
        "S04: [Error, The robot is experiencing an error.]",
        "S05: [Confused, The robot is confused and unsure what to do.]",
        "S06: [Unsure, It is unclear as the robot does not appear to be in any of the described states.]"
        ]
        
    elif state_space == 'minimized':
        set_of_states = [
        "S01: [Waiting for Input, The robot is in standby mode, waiting for a command from the user.]",
        "S02: [Interacting with Object, The robot is interacting with a target object in front of it on the ground.]", 
        "S03: [Needs Help, The robot is signalling that it needs help from the user.]",
        "S04: [Unsure, It is unclear as the robot does not appear to be in any of the described states.]"
        ]

        

# Used for Jackal robot
elif robot_string == 'jackal':
    if state_space == 'spread':
        set_of_states = [
            "S01: [Processing, The robot is calculating its navigation route to the delivery destination.]",
            "S02: [Navigating, The robot is navigating normally towards the delivery destination.]",
            "S03: [Danger, The robot is signaling for the user's attention due to a dangerous hazard.]",
            "S04: [Stuck, The robot is signaling for the user's attention as its wheel being stuck.]",
            "S05: [Accomplished, The robot is signaling that it has successfully reached the delivery destination.]",
            "S06: [Unsure, The robot's state is unclear as it does not appear to be in any of the described states.]"
        ]

    elif state_space == 'minimized':
        set_of_states = [
            "S01: [Progressing, The robot is progressing normally towards the delivery destination.]",
            "S02: [Alert, The robot is signaling for the user's attention due to a possible hazard or blocked path.]",
            "S03: [Accomplished, The robot is signaling that it has successfully reached the delivery destination.]",
            "S04: [Unsure, The robot's state is unclear as it does not appear to be in any of the described states.]"
        ]


state_legend_dict = {}

for state in set_of_states:
    code, rest = state.split(": ", 1)
    name = rest.split(",")[0].strip("[]")
    state_legend_dict[code] = name

print(state_legend_dict)


{'S01': 'Progressing', 'S02': 'Alert', 'S03': 'Accomplished', 'S04': 'Unsure'}


### Create Robot Instance | Set Assistant Prompt and Context

In [10]:
llm_assistant_prompt = "You are an expert roboticist and understand how to design communicative expressions for human-robot interaction."

robot_instance = robot_module.Robot(set_of_states)

if robot_string == 'go1':
    deployment_context = f"Consider a scenario where you are collaborating with a {robot_instance.form_factor} robot to locate and pick strawberries in a strawberry patch."

elif robot_string == 'jackal':
    deployment_context = f"Consider a scenario where you are collaborating with a {robot_instance.form_factor} robot to navigate through a warehouse and deliver packages to different locations."

In [11]:
robot_instance.get_action_space_shape()

(3, 3, 3, 3)

In [12]:
robot_instance.parameter_ranges

{'Beats Per Loop': 3, 'Pitch Bend': 3, 'Gain': 3, 'Distortion': 3}

## Accuracy Proxy Generator

### Generate the Accuracy Proxy .npy Array

In [13]:
# Initialize error and total counters
error_counter = 0
total_counter = 0

# Get action space shape, and use it to create a for loop that itterates all indices
action_space_shape = robot_instance.get_action_space_shape()

## SWAP || 
# Itterate through all possible actions in action space
count = 0
for indices in product(*(range(dim) for dim in action_space_shape)):
    count += 1
    robot_instance.set_active_parameter(list(indices))

    if printer:
        print(f"Action {count}:")
        print(robot_instance.active_parameters)
        print("\n\n")

    iteration_counter = 0
    iteration_error_counter = 0

    # Five iterations for loop
    for iteration in range(iteration_quantity):

        if printer:
            print(f"**********************************************************************")
            print(f"ITERATION {iteration+1} FOR {robot_instance.active_parameters}")
            print(f"**********************************************************************")

        # Generate description with test values and omission probability
        raw_description = robot_instance.generate_description(omission_probability)

        # Prepare a promt with context + robot description
        # summary_prompt = f"{deployment_context} \nSummarize the essential details about the robot's {robot_instance.communication_modality} from the text below. Do not add interpretations or explanations. Focus on summarizing the key aspects in the most direct way possible. \n{raw_description}"
        
        ### SWAP UNCOMMENT TO DISABLE TEST AND ENABLE LLM
        # Summarize the context + robot description using llm 
        summarized_expression = raw_description
        # summarized_expression = llm_prompt_reply(prompt=summary_prompt, 
        #                                           client=client, 
        #                                           llm_model=llm_model, 
        #                                           llm_assistant_prompt=llm_assistant_prompt, 
        #                                           temperature_coefficient=temperature_coefficient, 
        #                                           frequency_penalty_coefficient=frequency_penalty_coefficient, 
        #                                           top_p_coefficient=top_p_coefficient)


        # Print before and after summary
        if printer:
            print(f"\nRAW DESCRIPTION:\n{raw_description}")
            print(f"\nLLM SUMMARY (override same):\n{summarized_expression}")

        # Pass summarized expression to prompt builder, along with set of states and robot_instance parameters 
        acc_proxy_prompt = build_prompt(expression_string=summarized_expression, 
                                        state_list=set_of_states, 
                                        deployment_context=deployment_context, 
                                        llm_assistant_prompt=llm_assistant_prompt,
                                        expression_modality=robot_instance.communication_modality)
        
        if printer == "all":
            print(f"\n\nACCURACY PROXY PROMPT:\n~~~{acc_proxy_prompt}~~~\n\n")


        ### SWAP UNCOMMENT TO DISABLE TEST AND ENABLE LLM
        ## Use test data or run the prompt through the accuracy proxy model
        # six_state_test_options = ["[S01, State 01]", "[S02, State 02]", "[S03, State 03]", "[S04, State 04]", "[S05, State 05]", "[S06, State 06]"]
        # four_state_test_options = ["[S01, State 01]", "[S02, State 02]", "[S03, State 03]", "[S04, State 04]"]
        # acc_proxy_reply = random.choice(six_state_test_options)

        acc_proxy_reply = llm_prompt_reply(acc_proxy_prompt, client, llm_model, llm_assistant_prompt, temperature_coefficient, frequency_penalty_coefficient, top_p_coefficient)
        print(f"ACCURACY PROXY REPLY: {acc_proxy_reply}\n")

        # Parse the accuracy proxy reply to identify the state
        try:
            state_code, _ = acc_proxy_reply.strip("[]").split(", ")
            state_code = state_code.strip("'")
        except ValueError:
            print(f"Error: The GPT reply {acc_proxy_reply} was not in the correct format.")
            state_code = "E01"
        
        # Check if the state code exists within the dictionary
        if state_code in robot_instance.action_space[tuple(indices)]:
            # Increment the count for the identified state in the action space
            robot_instance.action_space[tuple(indices)][state_code] += 1
            iteration_counter += 1
            total_counter += 1

            if printer:
                # Output the identified state code
                print(f"Identified state code: {state_code}")
                # Output the updated action space
                print(f"Updated action space: {robot_instance.action_space[tuple(indices)]}\n")

        else:
            # Output an error message if the state code does not exist
            iteration_counter += 1
            iteration_error_counter += 1

            total_counter += 1
            error_counter += 1

            
            if printer:
                print(f"Error: State code '{state_code}' does not exist in the action space.\n")

        #     # Log the reply from the accuracy proxy model to the robot_instance action_space
        #     robot_instance.action_space[1, 1, 0, 0, 1, 0] = acc_proxy_reply
    
    if printer:
        print(f"@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        print(f"COMPLETED ALL {iteration_quantity} ITERATIONS FOR {robot_instance.active_parameters}\n")
        print(f"Iteration Error Count: {iteration_error_counter}\nIteration Count: {iteration_counter}")
        iteration_error_percentage = (iteration_error_counter / iteration_counter) * 100 if iteration_counter > 0 else 0
        print(f"Iteration Error Percentage: {iteration_error_percentage:.2f}%\n")

        print(f"Total Error Count: {error_counter}\nTotal Count: {total_counter}")
        total_error_percentage = (error_counter / total_counter) * 100 if total_counter > 0 else 0
        print(f"Total Error Percentage: {total_error_percentage:.2f}%")
        print(f"@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n\n")

if printer:
    print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    print(f"COMPLETED ALL COMBINATIONS")
    print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n")

Action 1:
[0 0 0 0]



**********************************************************************
ITERATION 1 FOR [0 0 0 0]
**********************************************************************

RAW DESCRIPTION:
The robot's beeps are clean with no distortion.

LLM SUMMARY (override same):
The robot's beeps are clean with no distortion.
ACCURACY PROXY REPLY: [S04, Unsure]

Identified state code: S04
Updated action space: {'S01': 0, 'S02': 0, 'S03': 0, 'S04': 1}

**********************************************************************
ITERATION 2 FOR [0 0 0 0]
**********************************************************************

RAW DESCRIPTION:
The robot beeps slowly 1 time per second. The pitch of the robot's beeps bend with a downward inflection.

LLM SUMMARY (override same):
The robot beeps slowly 1 time per second. The pitch of the robot's beeps bend with a downward inflection.
ACCURACY PROXY REPLY: [S04, Unsure]

Identified state code: S04
Updated action space: {'S01': 0, 'S02': 0, 'S

### Save RAW Array | Normalize | Save Scaled Array to External File

In [14]:
### Save RAW Array

# Define the save string for the raw numpy array
raw_accuracy_array_filename = robot_string + "_" + state_space + "_" + llm_model + "_" + attempt_ID + "ID_" + iteration_quantity_str + "ITR_raw.npy"
raw_accuracy_array_save_string = f"./../data/acc_arrays/{raw_accuracy_array_filename}"

# Save the numpy array robot_instance.action_space to a file 
print(f"Saving RAW numpy array to: {raw_accuracy_array_filename}")
np.save(f"{raw_accuracy_array_save_string}", robot_instance.action_space)


### Normalize Array
# Create a new numpy array for the normalized action space
normalized_action_space = np.zeros_like(robot_instance.action_space, dtype=dict)
for index in np.ndindex(robot_instance.action_space.shape):
    dict_obj = robot_instance.action_space[index]
    total = sum(dict_obj.values())  # Compute sum of values
    if total > 0:  # Avoid division by zero
        normalized_action_space[index] = {key: val / total for key, val in dict_obj.items()}



### Save NORMALIZED Array
# Print the name of the file where the numpy array will be saved
print(f"\nSaving NORMALIZED numpy array to: {normalized_accuracy_array_filename}")
np.save(f"{normalized_accuracy_array_save_string}", normalized_action_space)

# print first element of robot_instance.action_space and normalized_action_space
if robot_string == 'go1':           # 'spread' or 'minimized'
    print("RAW @ [(0, 0, 0, 0, 0, 0)]\t", robot_instance.action_space[(0, 0, 0, 0, 0, 0)])
    print("NORM @ [(0 ,0, 0, 0, 0, 0)]\t", normalized_action_space[(0, 0 ,0, 0, 0, 0)])

# print first element of robot_instance.action_space and normalized_action_space
elif robot_string == 'jackal':      # 'spread' or 'minimized'
    print("RAW @ [(0, 0, 0, 0)]\t", robot_instance.action_space[(0, 0, 0, 0)])
    print("NORM @ [(0, 0, 0, 0)]\t", normalized_action_space[(0, 0, 0, 0)])


Saving RAW numpy array to: jackal_minimized_gpt-4o_03ID_020ITR_raw.npy

Saving NORMALIZED numpy array to: jackal_minimized_gpt-4o_03ID_020ITR_norm.npy
RAW @ [(0, 0, 0, 0)]	 {'S01': 4, 'S02': 2, 'S03': 8, 'S04': 4}
NORM @ [(0, 0, 0, 0)]	 {'S01': 0.2222222222222222, 'S02': 0.1111111111111111, 'S03': 0.4444444444444444, 'S04': 0.2222222222222222}


## Evaluate the Cost Function

Here, we load the trajectory array generated using **trajectory_array_generator.py**, located in the `data/freedog` directory.

The `freedog` directory is a separate repository for the Unitree Go1 controller. 

For more details and access to the public repository, visit the repository: [github.com/liamreneroy/freedog](https://github.com/liamreneroy/freedog).

In [19]:
# SETTINGS

cost_function_weight = 0.7              # Weight for accuracy vs. distance
distance_method = "emd"                # distance metric (none, emd, dynamic ** GO1 ONLY **)
go1_max_dynamic_distance = 338.5619     # Previously calculated for Go1 case, used for dynamic distance normalization


In [20]:
# Enter the data in spreadsheet format
workbook_path = "./../data/selected_poses/optimization_outputs.xlsx"
response_book = load_workbook(workbook_path)



# Load accuracy proxy values from external file
ACCURACY_PROXY_FILE = normalized_accuracy_array_save_string    # File containing accuracy proxy values
TRAJECTORY_ARRAY_FILE = transformed_trajectory_save_string     # File containing trajectory array

loaded_accuracy_proxy_array = np.load(ACCURACY_PROXY_FILE, allow_pickle=True) # Load accuracy proxy values

if distance_method == 'dynamic':
    go1_transformed_trajectory_array = np.load(TRAJECTORY_ARRAY_FILE, allow_pickle=True)
else:
    go1_transformed_trajectory_array = None
                                       

# Get action space shape and create dict to store final poses
motion_param_ranges = robot_instance.get_action_space_shape()    # Example parameter ranges
poses_dict = {}                                                  # Dictionary to store selected poses


def get_accuracy(pose_a, state_code, accuracy_proxy_array):
    """
    Retrieves accuracy proxy value for a given pose A.
    """
    return accuracy_proxy_array[tuple(pose_a)][state_code]  # Access value based on indices in A


def get_min_distance(pose_a, poses_dict, trajectory_array=None, max_dynamic_distance=None):
    """
    Computes the minimum distance between pose_a and all defined expressions in poses_dict.
    """
    if poses_dict==None or len(poses_dict)==0:
        return 0.0
    
    distances = [calculate_distance(distance_type=distance_method, 
                                    action_space_shape=motion_param_ranges, 
                                    trajectory_array=trajectory_array, 
                                    max_dynamic_distance=max_dynamic_distance, 
                                    np_pose_a=np.array(pose_a), 
                                    np_pose_b=np.array(pose_B)) for pose_B in poses_dict.values()]
    return min(distances)


def evaluate_cost(pose_a, state_code, weight, poses_dict, accuracy_proxy_array, trajectory_array=None, max_dynamic_distance=None):
    """
    Evaluates the cost function f(pose_a, pose_a) using state_code.
    """
    accuracy_term = get_accuracy(pose_a=pose_a, 
                                 state_code=state_code,
                                 accuracy_proxy_array=accuracy_proxy_array)
    
    distance_term = get_min_distance(pose_a=pose_a, 
                                     poses_dict=poses_dict, 
                                     trajectory_array=trajectory_array, 
                                     max_dynamic_distance=max_dynamic_distance)
    
    cost = weight * accuracy_term + (1 - weight) * distance_term
    return cost, accuracy_term, distance_term


def find_best_pose(excel_sheet, 
                    attempt_id,
                    llm_model,
                    iteration_quantity,
                    robot_string,
                    state_space,
                    motion_param_ranges, 
                    state_code, 
                    distance_method,
                    weight, 
                    poses_dict, 
                    accuracy_proxy_array, 
                    trajectory_array=None, 
                    max_dynamic_distance=None):
    """
    Iterates through all possible parameter value combinations to find the best pose.
    """
    best_pose = None
    best_cost = float("-inf")
    iteration = 0
    columns = ["N", "O", "P", "Q", "R", "S"]


    for pose_a in product(*[range(v) for v in motion_param_ranges]):
        iteration += 1
        cost, accuracy_term, distance_term = evaluate_cost(pose_a=pose_a, 
                                                           state_code=state_code,
                                                           weight=weight, 
                                                           poses_dict=poses_dict, 
                                                           accuracy_proxy_array=accuracy_proxy_array,
                                                           trajectory_array=trajectory_array, 
                                                           max_dynamic_distance=max_dynamic_distance
                                                           )
        if cost > best_cost:
            best_cost = cost
            best_pose = pose_a
            print(f"Pose: {pose_a} | Accuracy: {accuracy_term:.3f} | Distance: {distance_term:.3f} | Cost: {cost:.3f} **********")

            excel_sheet["A"+str(iteration+1)] = attempt_id
            excel_sheet["B"+str(iteration+1)] = llm_model
            excel_sheet["C"+str(iteration+1)] = iteration_quantity
            excel_sheet["D"+str(iteration+1)] = robot_string
            excel_sheet["E"+str(iteration+1)] = state_space
            excel_sheet["F"+str(iteration+1)] = state_code
            excel_sheet["G"+str(iteration+1)] =str(pose_a)
            excel_sheet["H"+str(iteration+1)] = distance_method
            excel_sheet["I"+str(iteration+1)] = weight
            excel_sheet["J"+str(iteration+1)] = accuracy_term
            excel_sheet["K"+str(iteration+1)] = distance_term
            excel_sheet["l"+str(iteration+1)] = cost
            excel_sheet["M"+str(iteration+1)] = "*****"

            for i, col in enumerate(columns):
                try:
                    excel_sheet[f"{col}"+str(iteration+1)] = pose_a[i]
                except IndexError:
                    continue  # Handles cases where pose_a has fewer elements
                except Exception as e:
                    print(f"Error writing to {col}1: {e}")  # Debugging other potential issues

        else:
            # print(f"Pose: {pose_a} | Accuracy: {accuracy_term:.3f} | Distance: {distance_term:.3f} | Cost: {cost:.3f}")
    
            excel_sheet["A"+str(iteration+1)] = attempt_id
            excel_sheet["B"+str(iteration+1)] = llm_model
            excel_sheet["C"+str(iteration+1)] = iteration_quantity
            excel_sheet["D"+str(iteration+1)] = robot_string
            excel_sheet["E"+str(iteration+1)] = state_space
            excel_sheet["F"+str(iteration+1)] = state_code
            excel_sheet["G"+str(iteration+1)] =str(pose_a)
            excel_sheet["H"+str(iteration+1)] = distance_method
            excel_sheet["I"+str(iteration+1)] = weight
            excel_sheet["J"+str(iteration+1)] = accuracy_term
            excel_sheet["K"+str(iteration+1)] = distance_term
            excel_sheet["L"+str(iteration+1)] = cost
            excel_sheet["M"+str(iteration+1)] = ""
            
            for i, col in enumerate(columns):
                try:
                    excel_sheet[f"{col}"+str(iteration+1)] = pose_a[i]
                except IndexError:
                    continue  # Handles cases where pose_a has fewer elements
                except Exception as e:
                    print(f"Error writing to {col}1: {e}")  # Debugging other potential issues


    return best_pose, best_cost


# Iterate through all states to find the best pose for each state
print(f"\nCalculating Best Poses\nDistance Method: {distance_method}\nWeight: {cost_function_weight}\n")  

for state_code, state_name in state_legend_dict.items():
    if state_name == "Unsure":
        print("passing for state name:", state_name)
        continue

    if state_name in poses_dict: # add functionality for overwrite or keep
        old_pose = poses_dict[state_name]
        print(f"Overwriting {state_name}: {old_pose} with new pose.")

    sheet_name = (robot_string[:3] + "_" + state_space[:3] + "_" + distance_method[:3] + "_ID" + attempt_ID + "_" + state_code).upper()
    # print(f"\nState: {state_code}: {state_name} | Sheet: {sheet_name}\n")

    try: # Try to open existing sheet
        response_sheet = response_book[sheet_name]
    except KeyError:  # If ot doesn't exist. create it
        response_sheet = response_book.create_sheet(title=sheet_name)

    response_sheet["A1"] = "ID"
    response_sheet["B1"] = "model"
    response_sheet["C1"] = "iterations"
    response_sheet["D1"] = "robot"
    response_sheet["E1"] = "state space"
    response_sheet["F1"] = "state code"
    response_sheet["G1"] = "param combo"
    response_sheet["H1"] = "dist method"
    response_sheet["I1"] = "weight"
    response_sheet["J1"] = "accuracy"
    response_sheet["K1"] = "distance"
    response_sheet["L1"] = "cost"
    response_sheet["M1"] = "increased"
    response_sheet["N1"] = "P1"
    response_sheet["O1"] = "P2"
    response_sheet["P1"] = "P3"
    response_sheet["Q1"] = "P4"
    response_sheet["R1"] = "P5"
    response_sheet["S1"] = "P6"

    response_sheet["U1"] = "State"
    response_sheet["U2"] = "Result"
    response_sheet["U3"] = "Cost"


    # Find best pose for state
    best_pose, best_cost = find_best_pose(excel_sheet=response_sheet,
                                          attempt_id=attempt_ID,
                                          llm_model=llm_model,
                                          iteration_quantity=iteration_quantity,  
                                          robot_string=robot_string,
                                          state_space=state_space,
                                          motion_param_ranges=motion_param_ranges, 
                                          state_code=state_code,
                                          distance_method=distance_method,
                                          weight=cost_function_weight,
                                          poses_dict=poses_dict, 
                                          accuracy_proxy_array=loaded_accuracy_proxy_array,
                                          trajectory_array=go1_transformed_trajectory_array, 
                                          max_dynamic_distance=go1_max_dynamic_distance
                                          )
    
    print(f"Best Pose for {state_code}: {state_name}: {best_pose} with Cost: {best_cost:.3f}\n")
    
    response_sheet["V1"] = state_name
    response_sheet["V2"] = str(best_pose)
    response_sheet["V3"] = best_cost

    # Store best pose
    poses_dict[state_name] = best_pose

    # Save the spreadsheet before changing the sheet
    response_book.save(workbook_path)



Calculating Best Poses
Distance Method: emd
Weight: 0.7

Pose: (0, 0, 0, 0) | Accuracy: 0.222 | Distance: 0.000 | Cost: 0.156 **********
Pose: (0, 0, 1, 0) | Accuracy: 0.263 | Distance: 0.000 | Cost: 0.184 **********
Pose: (0, 1, 0, 0) | Accuracy: 0.750 | Distance: 0.000 | Cost: 0.525 **********
Pose: (0, 1, 1, 0) | Accuracy: 0.857 | Distance: 0.000 | Cost: 0.600 **********
Pose: (1, 2, 1, 0) | Accuracy: 0.941 | Distance: 0.000 | Cost: 0.659 **********
Best Pose for S01: Progressing: (1, 2, 1, 0) with Cost: 0.659

Pose: (0, 0, 0, 0) | Accuracy: 0.111 | Distance: 0.500 | Cost: 0.228 **********
Pose: (0, 0, 0, 1) | Accuracy: 0.250 | Distance: 0.625 | Cost: 0.363 **********
Pose: (0, 0, 0, 2) | Accuracy: 0.500 | Distance: 0.750 | Cost: 0.575 **********
Pose: (0, 0, 1, 2) | Accuracy: 0.579 | Distance: 0.625 | Cost: 0.593 **********
Pose: (0, 0, 2, 0) | Accuracy: 0.647 | Distance: 0.500 | Cost: 0.603 **********
Pose: (0, 0, 2, 1) | Accuracy: 0.800 | Distance: 0.625 | Cost: 0.747 **********

### Final Printout

In [17]:
print("PARAMETER LEGEND\n")
print("GO1: \nS01 | Body Direction \t[user, object]\nS02 | Body Tilt \t[left, neutral, right] \nS03 | Body Lean \t[backward, neutral, forward] \nS04 | Body Height \t[low, neutral, high] \nS05 | Mot. Smoothness \t[smooth, shaky] \nS06 | Mot. Velocity \t[slow, medium, fast]\n\n")

print("Jackal: \nS01 | Beats Per Loop \t[slow, medium, fast]\nS02 | Pitch Bend \t[down, neutral, up] \nS03 | Gain \t\t[quiet, neutral, loud] \nS04 | Distortion \t[none, mild, sharp]\n\n")

PARAMETER LEGEND

GO1: 
S01 | Body Direction 	[user, object]
S02 | Body Tilt 	[left, neutral, right] 
S03 | Body Lean 	[backward, neutral, forward] 
S04 | Body Height 	[low, neutral, high] 
S05 | Mot. Smoothness 	[smooth, shaky] 
S06 | Mot. Velocity 	[slow, medium, fast]


Jackal: 
S01 | Beats Per Loop 	[slow, medium, fast]
S02 | Pitch Bend 	[down, neutral, up] 
S03 | Gain 		[quiet, neutral, loud] 
S04 | Distortion 	[none, mild, sharp]




In [18]:
print(f"State Legend:")
for key, value in state_legend_dict.items():
    print(f"{key}:{value}")
print()

print("Parameter Legend:")
for i, key in enumerate(robot_instance.parameter_descriptions.keys(), start=1):
    print(f"P{i}: {key}")
print()

print(f"Final Poses:")
for key, value in poses_dict.items():
    print(f"{key}:{value}")
print()


State Legend:
S01:Progressing
S02:Alert
S03:Accomplished
S04:Unsure

Parameter Legend:
P1: Beats Per Loop
P2: Pitch Bend
P3: Gain
P4: Distortion

Final Poses:
Progressing:(1, 2, 1, 0)
Alert:(2, 0, 1, 2)
Accomplished:(0, 0, 0, 0)



### Final ToDo:
* Save to external file
